In [4]:
%matplotlib inline
from __future__ import print_function
import os
from pyspark import SQLContext
from pyspark.sql import Row
import pyspark.sql.functions as sql
#from pyspark.sql.functions import udf, length
import matplotlib.pyplot as plt
import numpy
import math
import matplotlib.pyplot as plt
import seaborn as sns

import nltk
import pyspark.ml.feature as feature

:0: FutureWarning: IPython widgets are experimental and may change in the future.


In [1]:
# Load Processed Parquet
sqlContext = SQLContext(sc)
notes = sqlContext.read.parquet("../data/idigbio_notes.parquet")
#idbdf = sqlContext.read.parquet("../data/idigbio-100k/occurrence.txt.parquet")
total_records = notes.count()
print(total_records)

3232459


In [2]:
# Small sample of the df
notes_sm = notes.sample(withReplacement=False, fraction=0.1)
notes_sm.cache()
print(notes_sm.count())

322952


In [5]:
# How much text is this?
print("Total text in MB")
print(notes_sm.select(sql.sum(notes_sm['document_len'])).collect()[0][0] / 1024^2)

Total text in MB
20100


Standard phases:
1. tokenize
1. remove stop words
1. stem
1. frequency hist
1. wordcloud

In [6]:
# This creates a new column with a list of tokens in it, not a column of tokens
tokenizer = feature.Tokenizer()
#print(tokenizer.params)
tokenizer.setParams(inputCol='document', outputCol='tokens')
notes_tokens = tokenizer.transform(notes_sm)
notes_tokens.head()

Row(uuid=u'57fc3a0d-6459-4bea-af3b-cf5b1bc0895b', occurrenceID=u'urn:lsid:biosci.ohio-state.edu:osuc_occurrences:OSAL__0004567', catalogNumber=u'OSAL 0004567', county=u'Hocking', institutionCode=u'Ohio State University Acarology Laboratory, Columbus, OH (OSAL)', country=u'United States', countryCode=u'', stateProvince=u'Ohio', family=u'', recordedBy=u'Gerdeman, B. S. (Beverly Swaim)', order=u'', specificEpithet=u'', genus=u'', sex=u'undetermined', scientificName=u'Narceolaps annularis', year=u'', month=u'', fieldNotes=u"[ USA: Ohio, Hocking Co. | Little Rocky Hollow| Gerdeman BS | 29 V 1998 | ex narceus | (Diplopoda: Spirobolidae) | BSG98-0529-4 AL5523 ] [ Iphiopsididae | Narceolaelaps annularis | Kethley 1978 | DN | det.: Gerdeman 1998 | Lactophenol | Hoyer's | OSAL0004567 ]", occurrenceRemarks=u'', eventRemarks=u'', document=u"  [ USA: Ohio, Hocking Co. | Little Rocky Hollow| Gerdeman BS | 29 V 1998 | ex narceus | (Diplopoda: Spirobolidae) | BSG98-0529-4 AL5523 ] [ Iphiopsididae | Na

In [7]:
# flatten to list of tokens & convert to dataframe
tokens = notes_tokens.flatMap(lambda x: x['tokens']).map(lambda x: (x, 1))
print(tokens.take(10))

# Below cells based on DF need this
#tokens = tokens.map(lambda w: Row(token=w)).toDF()
#print(tokens.head(10))

[(u'', 1), (u'', 1), (u'[', 1), (u'usa:', 1), (u'ohio,', 1), (u'hocking', 1), (u'co.', 1), (u'|', 1), (u'little', 1), (u'rocky', 1)]


In [8]:
# pyspark.ml.feature.StopWordsRemover

In [8]:
from wordcloud import WordCloud

In [11]:
#token_freq = tokens.groupBy('token').count()
#.sort(sql.col("count").sql.desc())
#print(token_freq.head(4))
#(group_by_dataframe
#    .count()
#    .filter("`count` >= 10")
#    .sort(col("count").desc()))
# This None NA bug stuff sucks.
#token_freq.show(5)

tokens.registerTempTable("tokens")
tokens_freq = sqlContext.sql("""
SELECT token, count(token) as c
FROM tokens
WHERE length(token) > 0 AND token IS NOT NULL
GROUP BY token
ORDER BY c DESC
LIMIT 100""")
#print(token_freq.show(4)) # Still null pointer exception!
#pd_tokens = tokens_freq.toPandas() # And happens outright here

In [9]:
# Back to using the RRD version of tokens
from operator import add
counts = tokens.reduceByKey(add)
print(counts.take(3))

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 11 in stage 9.0 failed 1 times, most recent failure: Lost task 11.0 in stage 9.0 (TID 1283, localhost): java.lang.NullPointerException
	at org.apache.spark.ml.feature.Tokenizer$$anonfun$createTransformFunc$1.apply(Tokenizer.scala:39)
	at org.apache.spark.ml.feature.Tokenizer$$anonfun$createTransformFunc$1.apply(Tokenizer.scala:39)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$SpecificUnsafeProjection.apply(Unknown Source)
	at org.apache.spark.sql.execution.Project$$anonfun$1$$anonfun$apply$1.apply(basicOperators.scala:51)
	at org.apache.spark.sql.execution.Project$$anonfun$1$$anonfun$apply$1.apply(basicOperators.scala:49)
	at scala.collection.Iterator$$anon$11.next(Iterator.scala:328)
	at scala.collection.Iterator$$anon$11.next(Iterator.scala:328)
	at org.apache.spark.api.python.SerDeUtil$AutoBatchedPickl